# morph-var 1

## Extract landform normal profiles from a DEM

This is the first of five notebooks outlining the algorithm we used to quantify the morphologic variability of a linear landform. In this notebook, we extract landform-normal profiles using three user-provided data: a DEM of the region of interest, a polygon shapefile that can be used to crop the DEM to the sepecific lanform if desired, and a line shapefile (contained within the cropping polygon) that traces the center of the landform.

The output of the notebook is a folder with text files for each landform-normal profile. Each text file has columns for the Easting, Northing, UTM Zone, and elevation of each point along the landform-normal profile. These text files are used as the data input for notebook 2.

### Packages and libraries

In [ ]:
import pandas as pd
import rioxarray as rxr
import geopandas as gpd
import numpy as np
from shapely.geometry import mapping,Point, LineString
import os
import rasterio
import pyosp
from osgeo import gdal  # import GDAL
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### Extract profile coordinates and store in text files

In [ ]:
# Define paths for DEM, cropping polygon shapefile, and landform trace line shapefile
#dem path
dem_clipped_path =  r"./DEM/dem7.tif"
#Geomorphic trajectory total element file, providing coordinate system
landform_trace_path = r"./DEM/GIF1.shp"
#dem resolution ratio
dem_resolution=400
#Folder for all individual terrain trajectories shp
root_path=r"./land_file"
#Obtain coordinate system
crop_polygon = gpd.read_file(landform_trace_path)
###Cycle each landform trajectory
for root,dirs,files in os.walk(root_path):
    for file in files:
        if file.endswith(".shp"):
            landform_name=os.path.splitext(file)[0]
            landform_trace_path=os.path.join(root,file)
            try:
                #profiling
                elev = pyosp.Elev_curv(landform_trace_path, dem_clipped_path, width=1000,min_elev=-5319,max_elev=8415,
                        line_stepsize=dem_resolution,cross_stepsize=None)
                #save jpg
                fig, ax = plt.subplots()
                elev.profile_plot(ax=ax, color="maroon")
                fig.savefig("C:/Users/user/Desktop/morph_var1/fig/"+landform_name+".jpg")
            except:
                with open('C:/Users/user/Desktop/morph_var1/log.txt','a') as f:
                    f.write(file+"The trajectory is too small to draw a profile\n")
                continue
            path_shapefile="C:/Users/user/Desktop/morph_var1/profile_shapefile/profile_lines"+file
            profile_lines=[]
            ###Draw trajectory contour map and generate shp
            for a in range(0,len(elev.lines)):
                line_dat=elev.lines[a]
                print(line_dat)
                coords=[(coord[0], coord[1]) for coord in line_dat]
                print(coords)
                #points= [Point(coord[0], coord[1]) for coord in line_dat]
                lines=LineString(coords)
                profile_lines.append(lines)
            prof_number=np.array(range(0,len(elev.lines)))
            distance=prof_number*dem_resolution

            d = {'col1': prof_number,'col2': distance, 'geometry': profile_lines}
            df=gpd.GeoDataFrame(d,geometry='geometry',crs=crop_polygon.crs)
            df.to_file(path_shapefile)
            len(df)
            path="C:/Users/user/Desktop/morph_var1/"+landform_name+'_raw_profiles/'
            if os.path.exists(path)==False: 
                os.mkdir(path)
            profile_numbers=list(range(0,len(elev.distance)))

            ###Save coordinates as a txt file
            profile_filenames=[landform_name+'_'+f'{a}'.zfill(6)+'.txt' for a in profile_numbers]

            # Save profile files in newly created folders. The information is stored in 4 columns: Easting coordinate (m), Westing coordinate (m), elevation (m) and CRS (espg code).
            for a in profile_numbers:
                elevation_temp = elev.dat[a]
                x_coord_temp=[elev.lines[a][b][0] for b in list(range(0,len(elev.lines[a])))]
                y_coord_temp=[elev.lines[a][b][1] for b in list(range(0,len(elev.lines[a])))]
                
                x_coord=pd.Series(x_coord_temp)
                y_coord=pd.Series(y_coord_temp)
                elevation=pd.Series(elevation_temp)
                
                landform_profile_info = pd.concat([x_coord,y_coord,elevation],axis=1)
                landform_profile_info.columns=('Easting','Westing','elevation')
                
                landform_profile_crs=pd.Series(crop_polygon.crs,name='CRS')
                landform_profile_info=pd.concat([landform_profile_info,landform_profile_crs],axis=1)
                
                file=profile_filenames[a]
                landform_profile_info.to_csv(path+file)